﻿
# 实战项目 1：基于动量的交易策略
## 说明
每个问题都包含要实现的函数和如何实现该函数的说明。`# TODO` 注释标注了函数中需要实现的部分。实现函数后，请运行单元格，以根据我们提供的单元测试检验该函数。对于每个问题，我们都在 `project_tests` 包中提供了一个或多个单元测试。这些单元测试不会提示你的答案是否正确，但是如果有任何重大错误，会警告你。当你提交项目后，我们会检查你的解决方案是否正确。

## 软件包
在实现函数时，你只需使用在教室中用到的包，例如 [Pandas](https://pandas.pydata.org/) 和 [Numpy](http://www.numpy.org/)。我们将为你导入这些包。建议不要添加任何导入语句，否则评分者可能无法运行你的代码。

我们导入的其他包为 `helper`、`project_helper` 和 `project_tests`。这些是专门帮助你解决问题的自定义包。`helper` 和 `project_helper` 模块包含实用程序和图形函数。`project_tests` 包含所有问题的单元测试。

### 安装包

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### 加载包

In [ ]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## 市场数据
### 加载数据
我们在此项目中经常用到的是当日结束数据。其中包含很多股票的数据，但是我们将查看的是标普 500 中的股票。为了便于操作，我们缩小了日期范围，没有使用所有的数据。

In [ ]:
df = pd.read_csv('../../data/project_1/eod-quotemedia.csv', parse_dates=['date'], index_col=False)

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

print('Loaded Data')

### 股票示例
我们看看单支股票的收盘价看起来是怎样的。对于此示例及项目中的其他示例，我们将使用 Apple 的股票 (AAPL)。如果绘制所有股票的图形，那么信息太多了。

In [ ]:
apple_ticker = 'AAPL'
project_helper.plot_stock(close[apple_ticker], '{} Stock'.format(apple_ticker))

## 重采样调整后的价格

在此项目中，你要构建的交易信号不需要基于每日价格，例如，你可以使用月底价格每月进行一次交易。首先需要将每日调整后收盘价重采样为按月划分，并选择每个月最后的观察值。

实现 `resample_prices` 以按照频率 `freq` 重采样 `close_prices`。

In [ ]:
def resample_prices(close_prices, freq='M'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_resample_prices(resample_prices)

### 查看数据
将此函数应用到 `close` 上并查看结果。

In [ ]:
monthly_close = resample_prices(close)
project_helper.plot_resampled_prices(
    monthly_close.loc[:, apple_ticker],
    close.loc[:, apple_ticker],
    '{} Stock - Close Vs Monthly Close'.format(apple_ticker))

## 计算对数收益率

根据价格 ($P_t$) 计算对数收益率 ($R_t$)，并作为主要动量指标：

$$R_t = log_e(P_t) - log_e(P_{t-1})$$

实现下面的 `compute_log_returns` 函数，使其能接受 dataframe（例如 `resample_prices` 返回的 dataframe），并生成类似的对数收益率 dataframe。使用 Numpy 的[对数函数](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html)计算对数收益率。

In [ ]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_compute_log_returns(compute_log_returns)

### 查看数据
使用 `resample_prices` 返回的数据生成对数收益率。

In [ ]:
monthly_close_returns = compute_log_returns(monthly_close)
project_helper.plot_returns(
    monthly_close_returns.loc[:, apple_ticker],
    'Log Returns of {} Stock (Monthly)'.format(apple_ticker))

## 偏移收益率
实现 `shift_returns` 函数，使对数收益率偏移为时间序列中的之前或未来收益率。例如，参数 `shift_n` 为2，`returns` 如下所示：

```
                           Returns
               A         B         C         D
2013-07-08     0.015     0.082     0.096     0.020     ...
2013-07-09     0.037     0.095     0.027     0.063     ...
2013-07-10     0.094     0.001     0.093     0.019     ...
2013-07-11     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...
```

`shift_returns` 函数的输出应该为：
```
                        Shift Returns
               A         B         C         D
2013-07-08     NaN       NaN       NaN       NaN       ...
2013-07-09     NaN       NaN       NaN       NaN       ...
2013-07-10     0.015     0.082     0.096     0.020     ...
2013-07-11     0.037     0.095     0.027     0.063     ...
...            ...       ...       ...       ...
```

使用和上面相同的 `returns` 数据，但是 `shift_n` 为-2，`shift_returns` 函数应该生成以下结果：
```
                        Shift Returns
               A         B         C         D
2013-07-08     0.094     0.001     0.093     0.019     ...
2013-07-09     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...       ...
...            ...       ...       ...       ...       ...
...            NaN       NaN       NaN       NaN       ...
...            NaN       NaN       NaN       NaN       ...
```

_注意：“...”表示我们没有显示的数据。_

In [ ]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_shift_returns(shift_returns)

### 查看数据
我们来获取上个月和下个月的收益率。

In [ ]:
prev_returns = shift_returns(monthly_close_returns, 1)
lookahead_returns = shift_returns(monthly_close_returns, -1)

project_helper.plot_shifted_returns(
    prev_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Previous Returns of {} Stock'.format(apple_ticker))
project_helper.plot_shifted_returns(
    lookahead_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Lookahead Returns of {} Stock'.format(apple_ticker))

## 生成交易信号

交易信号是一系列交易操作，或可以用来采取交易操作的结果。常见的形式是在每个日期（例如在每个月底或以你认为合适的频率）生成“多头”和“空头”股票投资组合。这种信号可以用于在每个这样的日期再平衡投资组合，根据信号指示做多（买入）或做空（卖出）。

我们将尝试以下策略：
> 对于每个月底观察时期，按照上一个收益率从高到低对股票排序。选择表现靠前的股票作为多头，并选择表现靠后的股票作为空头。

实现 `get_top_n` 函数以获取每个月表现靠前的股票。从 `prev_returns` 中获取表现靠前的股票并为它们分配值 1。对于所有其他股票，分配值 0。例如，使用以下 `prev_returns`：

```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0.015     0.082     0.096     0.020     0.075     0.043     0.074
2013-07-09     0.037     0.095     0.027     0.063     0.024     0.086     0.025
...            ...       ...       ...       ...       ...       ...       ...
```

将 `top_n` 设为 3 时，函数 `get_top_n` 应该返回以下值：
```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0         1         1         0         1         0         0
2013-07-09     0         1         0         1         0         1         0
...            ...       ...       ...       ...       ...       ...       ...
```

*注意在实现该函数时，你可能需要使用 Panda 的 [`DataFrame.iterrows`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.iterrows.html) 和[`Series.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nlargest.html)。在这种情形下创建向量化解决方案很难。*

In [ ]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    # TODO: Implement Function
    
    return None

project_tests.test_get_top_n(get_top_n)

### 查看数据
我们希望获得表现很好和表现很差的股票。我们将使用 `get_top_n` 函数获得表现很好的股票。同样使用 `get_top_n` 函数获得表现很差的股票。但是，我们将传入 `-1*prev_returns` 而不是 `prev_returns`。乘以 -1 将使正收益率变成负收益率，反之亦然。所以它将返回表现很差的股票。

In [ ]:
top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)
project_helper.print_top(df_long, 'Longed Stocks')
project_helper.print_top(df_short, 'Shorted Stocks')

## 预测收益率
接下来我们将检测交易信号是否能获利。

首先计算此投资组合能够获得的净收益率。为了简单起见，我们假设每支股票的投资金额一样。这样可以将投资组合的收益率简化为每支股票收益率的算术平均值。

实现 `portfolio_returns` 函数以计算预期投资组合收益率。使用 `df_long` 表示做多的股票，并使用 `df_short` 表示做空的股票，使用 `lookahead_returns` 计算收益率。为了便于计算，我们向你提供了 `n_stocks` 作为股票数量，表示在单个时期内投资的股票数量。

In [ ]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_portfolio_returns(portfolio_returns)

### 查看数据
看看投资组合的效果。

In [ ]:
expected_portfolio_returns = portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)
project_helper.plot_returns(expected_portfolio_returns.T.sum(), 'Portfolio Returns')

## 统计学检验
### 年化收益率

In [ ]:
expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))

年化收益率使我们能够比较此策略的收益率与其他收益率，这些收益率通常按年计。 

### T 检验
我们的零假设 ($H_0$) 是交易信号的实际平均收益率为零。我们将对观察到的平均收益率执行单样本、单尾 t 检验，看看能否拒绝 $H_0$。

首先需要计算 t 统计量，然后算出对应的 p 值。p 值表示观察到与零假设为真时一样极端或比其更极端的 t 统计量的概率是多少。p 值很小表示观察到在零假设下观察到的 t 统计量的概率很小，所以引起对零假设的怀疑。在计算 p 值之前，建议设置一定的置信水平或 alpha ($\alpha$)，如果 $p < \alpha$，则拒绝零假设。

对于此项目，$\alpha = 0.05$，这是很常见的值。

实现 `analyze_alpha` 函数以对投资组合收益率样本进行t 检验。我们已经为你导入了 `scipy.stats` 模块。

注意：[`scipy.stats.ttest_1samp`](https://docs.scipy.org/doc/scipy-1.0.0/reference/generated/scipy.stats.ttest_1samp.html) 会执行双侧检验，所以将 p 值除以 2 可获得单侧p 值。

In [ ]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    # TODO: Implement Function

    return None

project_tests.test_analyze_alpha(analyze_alpha)

### 查看数据
我们看看我们的投资组合会获得什么样的值。运行完毕后，请回答以下问题。

In [ ]:
t_value, p_value = analyze_alpha(expected_portfolio_returns_by_date)
print("""
Alpha analysis:
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(t_value, p_value))

### 问题：你的 p 值是多少？这表示你的信号效果如何？

*#TODO:请在此单元格中输入答案*

## 提交
项目完成后，就可以提交项目了。请点击右下角的提交按钮。我们的审阅专家会对你的项目提供反馈，并指出是否通过评估。在等待反馈期间，你可以继续转到下个部分。